In [50]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords, brown
from nltk.stem import PorterStemmer, WordNetLemmatizer
from bs4 import BeautifulSoup
import re
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt


from keras.layers import Embedding, LSTM, Dense, Conv1D, MaxPooling1D, Dropout, Activation
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import numpy as np

import gzip
import json
import numpy as np
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.svm import SVC
from sklearn.metrics import mean_absolute_error, mean_squared_error
# Fit and transform the training data to a document-term matrix using TfidfVectorizer 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from sklearn.metrics import precision_score, recall_score, make_scorer
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import mean_absolute_error, mean_squared_error

import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier

# Data Preparation

data = pd.read_json('AMAZON_FASHION.json', lines=True)
data

In [ ]:
data = []
with gzip.open(os.path.join(f'CDs_and_Vinyl.json.gz')) as f:
        for l in f:
            data.append(json.loads(l.strip()))

In [ ]:
data

In [ ]:
data.info()

In [ ]:
data.shape

In [18]:
min_amount_product_mentions = 20
min_amount_user_mentions = 20
filtered_df = data[['asin', 'reviewerID', 'overall', 'reviewText']]
    
filtered_df = filtered_df.drop_duplicates()    
filtered_df = filtered_df[filtered_df['asin'].map(filtered_df['asin'].value_counts()) >= min_amount_product_mentions]
filtered_df = filtered_df[filtered_df['reviewerID'].map(filtered_df['reviewerID'].value_counts()) >= min_amount_user_mentions]

In [19]:
df = filtered_df[~filtered_df.reviewText.isna()]
df.rename(columns={'reviewText':'sentence'}, inplace=True)
print(len(df))

11522


C:\Users\emanu\anaconda3\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [20]:
STOPWORDS = set(stopwords.words('english'))
MIN_WORDS = 4
MAX_WORDS = 200

PATTERN_S = re.compile("\'s")  # matches `'s` from text  
PATTERN_RN = re.compile("\\r\\n") #matches `\r` and `\n`
PATTERN_PUNC = re.compile(r"[^\w\s]") # matches all non 0-9 A-z whitespace 

def clean_text(text):
    """
    Series of cleaning. String to lower case, remove non words characters and numbers.
        text (str): input text
    return (str): modified initial text
    """
    text = text.lower()  # lowercase text
    text = re.sub(PATTERN_S, ' ', text)
    text = re.sub(PATTERN_RN, ' ', text)
    text = re.sub(PATTERN_PUNC, ' ', text)
    return text

def tokenizer(sentence, min_words=MIN_WORDS, max_words=MAX_WORDS, stopwords=STOPWORDS, lemmatize=True):
    """
    Lemmatize, tokenize, crop and remove stop words.
    """
    if lemmatize:
        stemmer = WordNetLemmatizer()
        tokens = [stemmer.lemmatize(w) for w in word_tokenize(sentence)]
    else:
        tokens = [w for w in word_tokenize(sentence)]
    token = [w for w in tokens if (len(w) > min_words and len(w) < max_words
                                                        and w not in stopwords)]
    return tokens    


def clean_sentences(df):
    """
    Remove irrelavant characters (in new column clean_sentence).
    Lemmatize, tokenize words into list of words (in new column tok_lem_sentence).
    """
    print('Cleaning sentences...')
    df['clean_sentence'] = df['sentence'].apply(clean_text)
    df['tok_lem_sentence'] = df['clean_sentence'].apply(
        lambda x: tokenizer(x, min_words=MIN_WORDS, max_words=MAX_WORDS, stopwords=STOPWORDS, lemmatize=True))
    return df
    


In [22]:
df = clean_sentences(df)

Cleaning sentences...


C:\Users\emanu\AppData\Local\Temp/ipykernel_45576/3648348919.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['clean_sentence'] = df['sentence'].apply(clean_text)
C:\Users\emanu\AppData\Local\Temp/ipykernel_45576/3648348919.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tok_lem_sentence'] = df['clean_sentence'].apply(


In [23]:
print(len(df))
df[['sentence', 'clean_sentence', 'tok_lem_sentence']]

11522


,sentence,clean_sentence,tok_lem_sentence
2218,"My babies are big for their age, I have a 3 mo...",my babies are big for their age i have a 3 mo...,"[my, baby, are, big, for, their, age, i, have,..."
2284,My friend who works on a reservation loved the...,my friend who works on a reservation loved the...,"[my, friend, who, work, on, a, reservation, lo..."
2698,Good,good,[good]
2713,Good for a day,good for a day,"[good, for, a, day]"
2759,The GF likes these. She can ware them out of t...,the gf likes these she can ware them out of t...,"[the, gf, like, these, she, can, ware, them, o..."
...,...,...,...
882402,Very nice set!! These nose rings looked great ...,very nice set these nose rings looked great ...,"[very, nice, set, these, nose, ring, looked, g..."
882509,This shirt is nice.... The material is not to ...,this shirt is nice the material is not to ...,"[this, shirt, is, nice, the, material, is, not..."
882748,These broke really fast. A screw came out the ...,these broke really fast a screw came out the ...,"[these, broke, really, fast, a, screw, came, o..."
883160,looks nice,looks nice,"[look, nice]"


In [36]:
X = df['sentence']
y = df['overall']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

# SVM

In [28]:
pipeline = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SVC())])

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)

print("RMSE:", metrics.mean_squared_error(y_test, y_pred, squared=False))
print("Mean Squared Error",mean_squared_error(y_test, y_pred))

RMSE: 0.9329565199374099
Mean Squared Error 0.8704078680937228


In [40]:
def review_classification(X, y, model,clf_model):
    
    X_c = model.fit_transform(X)
    #print(X_c)
    print('# features: {}'.format(X_c.shape[1]))
    X_train, X_test, y_train, y_test =  train_test_split(X_c, y, test_size=0.3, stratify=y, random_state=42)
    print('# train records: {}'.format(X_train.shape[0]))
    print('# test records: {}'.format(X_test.shape[0]))
    
    
    
    clf = clf_model.fit(X_train, y_train)
    acc = clf.score(X_test, y_test)
    print("Mean Squared Error",mean_squared_error(y_test, y_pred))
    print("RMSE:", metrics.mean_squared_error(y_test, y_pred, squared=False))
    print ('Model Accuracy: {}'.format(acc))

In [43]:
review_classification(X, y, c, SVC())

# features: 9084
# train records: 8065
# test records: 3457
Mean Squared Error 0.8704078680937228
RMSE: 0.9329565199374099
Model Accuracy: 0.6977147816025455


In [44]:
review_classification(X, y, c, MultinomialNB())

# features: 9084
# train records: 8065
# test records: 3457
Mean Squared Error 0.8704078680937228
RMSE: 0.9329565199374099
Model Accuracy: 0.6632918715649407


In [47]:
review_classification(X, y, c, KNeighborsClassifier())

# features: 9084
# train records: 8065
# test records: 3457
Mean Squared Error 0.8704078680937228
RMSE: 0.9329565199374099
Model Accuracy: 0.658952849291293


In [48]:
review_classification(X, y, c, LogisticRegression())


# features: 9084
# train records: 8065
# test records: 3457
Mean Squared Error 0.8704078680937228
RMSE: 0.9329565199374099
Model Accuracy: 0.7089962395140295


C:\Users\emanu\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
